In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import tensorflow as tf
import pandas as pd
import seaborn as sns



In [ ]:
!#unzip '/content/drive/MyDrive/onedrive image/OneDrive_2021-09-03.zip'

In [ ]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
pathname = "/content/6659 images"
print("loading images...")
imagePaths = list(paths.list_images(pathname))
data = []
labels = []

loading images...


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# loop over the image paths
for imagePath in imagePaths:
	# extract the class label from the filename
	label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
	image = load_img(imagePath,target_size = (224,224))
	image = img_to_array(image)
	image = preprocess_input(image)

	# update the data and labels lists, respectively
	data.append(image)
	labels.append(label)

# convert the data and labels to NumPy arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# perform one-hot encoding on the labels



/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
labels

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
#labels.todense()

matrix([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])

In [ ]:
data.shape

(6659, 224, 224, 3)

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing


(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, stratify=labels, random_state=43)

# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(3, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [ ]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

# compile our model
print("compiling model...")
opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

compiling model...


In [ ]:
#trainY

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
# train the head of the network
print("training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	#validation_steps=len(testX) // BS,
	epochs=EPOCHS)

training head...
Epoch 1/20
157/157 [==============================] - 101s 447ms/step - loss: 0.6150 - accuracy: 0.7505 - val_loss: 0.2185 - val_accuracy: 0.9369
Epoch 2/20
157/157 [==============================] - 69s 439ms/step - loss: 0.2843 - accuracy: 0.8993 - val_loss: 0.1436 - val_accuracy: 0.9562
Epoch 3/20
157/157 [==============================] - 66s 422ms/step - loss: 0.2090 - accuracy: 0.9265 - val_loss: 0.1163 - val_accuracy: 0.9646
Epoch 4/20
157/157 [==============================] - 67s 427ms/step - loss: 0.1646 - accuracy: 0.9417 - val_loss: 0.0989 - val_accuracy: 0.9706
Epoch 5/20
157/157 [==============================] - 65s 415ms/step - loss: 0.1519 - accuracy: 0.9487 - val_loss: 0.0898 - val_accuracy: 0.9718
Epoch 6/20
157/157 [==============================] - 66s 423ms/step - loss: 0.1317 - accuracy: 0.9529 - val_loss: 0.0836 - val_accuracy: 0.9736
Epoch 7/20
157/157 [==============================] - 65s 413ms/step - loss: 0.1166 - accuracy: 0.9614 - val_los

In [ ]:
# make predictions on the testing set
print("evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("saving face mask detector model...")
model.save("/content/drive/MyDrive/face_mask_detector_1", save_format="h5")

evaluating network...
               precision    recall  f1-score   support

improper_mask       0.97      1.00      0.98       555
    with_mask       0.98      0.98      0.98       555
 without_mask       0.99      0.97      0.98       555

     accuracy                           0.98      1665
    macro avg       0.98      0.98      0.98      1665
 weighted avg       0.98      0.98      0.98      1665

saving face mask detector model...


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
